In [65]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [66]:
def func(x):
    a = 1.01
    b = -3.4
    c = 2.07
    return a*x**2 + b*x + c

In [67]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x

In [68]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h)+f(x))

In [69]:
def trapezoid_method(f,a,b,N):
    #f == fucntion to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evaluations to use
    #note the number of chunks will be N-1
    #so if N is odd, then we don't need to adjust the
    #last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        #print(i,i+1,x[i],x[i]+h,x[-2],x[-1])
        
    #return the answer
    return Fint

In [70]:
def simpson_core(f,x,h):
    return h*(f(x) + 4*f(x+h) + f(x+2*h))/3.

In [71]:
def simpsons_method(f,a,b,N):
    #f == fucntion to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evaluations to use
    #note the number of chunks will be N-1
    #so if N is odd, then we don't need to adjust the
    #last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #if N is odd or even, we have different
    #numbers of chunks to add
    if((N%2)==0):
        lim = 3
    else:
        lim = 2 
        
    #perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        #print(i,i+1,x[i],x[i]+h,x[-2],x[-1])
        
    #apply simpson's rule over the last integral
    #if N is even
    if((N%2)==0):
        Fint += simpson_core(f,x[-2],0.5*h)
        
        
    return Fint

In [72]:
def romberg_core(f,a,b,I,i):
    
    #we need the difference b-a
    h = b-a
    
    #and the increment between ne func evals
    dh = h/2.**(i)
    
    #we need the cofactor
    K = h/2.**(i+1)
    
    #and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #return the answer
    return K*M

In [73]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i = 0
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error estimate, set to a larger value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a)+f(b)) 
    
    #iterate by 1 
    i += 1
    
    while(delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1]+ romberg_core(f,a,b,I,i)
        
        #compute the new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i] )
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate
            i += 1
            
            #if we've reached the maximum iterations
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after',i)
                
    #return the answer
    return I[i]

In [74]:
Answer = func_integral(1)-func_integral(0)
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpson's Method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-4
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)

0.8866666666666665
Trapezoid
0.708744855967078
Simpson's Method
0.6821696387745769
Romberg
1 0.7803125 0.875 0.12134561473768528
2 0.7408593749999999 0.7803125 0.05325318991880219
3 0.7231054687499998 0.7408593749999999 0.02455230532371778
4 0.7147216796874998 0.7231054687499998 0.011730145175064101
5 0.7106530761718749 0.7147216796874998 0.005725161336867088
6 0.7086495971679686 0.7106530761718749 0.0028271786393627442
7 0.7076555633544915 0.7086495971679686 0.001404685930490056
8 0.7071604728698726 0.7076555633544915 0.0007001105175034796
9 0.7069134092330922 0.7071604728698726 0.00034949632239743403
10 0.7067899978160866 0.7069134092330922 0.00017460832409469342
11 0.7067283222079253 0.7067899978160866 8.726918990413394e-05
0.7067283222079253 -0.2029379824722646 0.0001
